In [ ]:
#Открываем файл и переводим в таблицу
import pandas as pd
import numpy as np
df = pd.read_csv('discounts_raw.csv', delimiter= ',')
df.head()

In [ ]:
#Открываем файл и переводим в таблицу
df_sales = pd.read_csv('sales_raw.csv', delimiter= ',')
df_sales.head()

In [ ]:
#разбиваем даты акций на год и месяц и переводим их в числа
df_date_year_and_month = df.copy()
df_date_year_and_month['date_start'] = df_date_year_and_month['date_start'].astype(str)
df_date_year_and_month[['yy_start','mm_start','dd']] = df_date_year_and_month['date_start'].str.split('-', expand = True)
df_date_year_and_month['yy_start'] = pd.to_numeric(df_date_year_and_month['yy_start'])
df_date_year_and_month['mm_start'] = pd.to_numeric(df_date_year_and_month['mm_start'])
df_date_year_and_month = df_date_year_and_month.drop(['dd'], axis = 1)


df_date_year_and_month['date_end'] = df_date_year_and_month['date_end'].astype(str)
df_date_year_and_month[['yy_end','mm_end','dd']] = df_date_year_and_month['date_end'].str.split('-', expand = True)
df_date_year_and_month['yy_end'] = pd.to_numeric(df_date_year_and_month['yy_end'])
df_date_year_and_month['mm_end'] = pd.to_numeric(df_date_year_and_month['mm_end'])
df_date_year_and_month = df_date_year_and_month.drop(['dd'], axis = 1)
df_date_year_and_month = df_date_year_and_month.drop(['date_end','date_start'], axis = 1)
df_date_year_and_month.head()

In [ ]:
# Избавляемся от пустых значений
df_date_year_and_month =df_date_year_and_month.dropna(axis=0)
df_date_year_and_month

In [ ]:
# Здесь мы видим, что нашы даты теперь int
df_date_year_and_month.info()

In [ ]:
#разбиваем даты продаж на год и месяц и переводим их в числа
df_sales_year_and_month = df_sales.copy()
df_sales_year_and_month['date'] = df_sales_year_and_month['date'].astype(str)
df_sales_year_and_month[['yy','mm','dd']] = df_sales_year_and_month['date'].str.split('-', expand = True)
df_sales_year_and_month['yy'] = pd.to_numeric(df_sales_year_and_month['yy'])
df_sales_year_and_month['mm'] = pd.to_numeric(df_sales_year_and_month['mm'])
df_sales_year_and_month['dd'] = pd.to_numeric(df_sales_year_and_month['dd'])
df_sales_year_and_month = df_sales_year_and_month.drop(['date'], axis = 1)
df_sales_year_and_month

In [ ]:
#фильтруем их по месяцу и году
df_sales_year_and_month = df_sales_year_and_month.loc[(df_sales_year_and_month['yy'] == 2023) &(df_sales_year_and_month['mm'] ==12)]
df_sales_year_and_month

In [ ]:
#совмещаем их в одну таблицу
df_big = pd.merge(df_sales_year_and_month, df_date_year_and_month, on='item_id', how='inner')
df_big['sale'] = np.where(((df_big['mm_end']<12) | (df_big['yy_end'] ==2023)| df_big['mm']==12), df_big['sale_price_before_promo'], df_big['sale_price_time_promo'])
df_big = df_big.drop(['sale_price_before_promo'], axis = 1)
df_big = df_big.drop(['sale_price_time_promo'], axis = 1)
df_big = df_big.drop_duplicates(subset=['dd','item_id'])

df_b = df_big[df_big['mm'] == 12]
df_new = df_b.groupby('item_id')['qnty'].sum().reset_index()
df_new

In [ ]:
#модель ARIMA

import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

# Загрузка данных
df_big = pd.merge(df_sales_year_and_month, df_date_year_and_month, on='item_id', how='inner')
df_big['sale'] = np.where(((df_big['mm_end']<12) | (df_big['yy_end'] ==2023)| df_big['mm']==12), df_big['sale_price_before_promo'], df_big['sale_price_time_promo'])
df_big = df_big.drop(['sale_price_before_promo', 'sale_price_time_promo'], axis=1)
df_big = df_big.drop_duplicates(subset=['dd','item_id'])
df_filtered = df_big[df_big['mm'] == 12]
df_new = df_filtered.groupby('item_id')['qnty'].sum().reset_index()


january_2024 = df_new.copy()

# Подготовка данных 
data = df_big[['dd', 'qnty']].groupby('dd').sum()

# Разделение данных на обучающий и тестовый наборы
train_size = int(len(data) * 0.8)  
train_data, test_data = data.iloc[:train_size], data.iloc[train_size:]

# Обучение модели 
model = ARIMA(train_data, order=(5,1,0))  
model_fit = model.fit()

# Прогнозирование продаж на январь 2024 года
forecast = model_fit.forecast(steps=len(test_data))  
january_2024['прогноз'] = forecast
january_2024.fillna(0, inplace=True)

# Запись прогноза в CSV
january_2024.to_csv('ARIMA_jan_2024.csv', sep =';',index=False, header=False, lineterminator=';\n')

# Оценка качества прогноза с использованием RMSE
rmse = sqrt(mean_squared_error(test_data, forecast))
print(f'RMSE прогноза на январь 2024: {rmse}')

In [ ]:
#модель Random forest
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# Загрузка данных
df_big = pd.merge(df_sales_year_and_month, df_date_year_and_month, on='item_id', how='inner')
df_big['sale'] = np.where(((df_big['mm_end']<12) | (df_big['yy_end'] ==2023)| df_big['mm']==12), df_big['sale_price_before_promo'], df_big['sale_price_time_promo'])
df_big = df_big.drop(['sale_price_before_promo', 'sale_price_time_promo'], axis=1)
df_big = df_big.drop_duplicates(subset=['dd','item_id'])
df_filtered = df_big[df_big['mm'] == 12]
df_new = df_filtered.groupby('item_id')['qnty'].sum().reset_index()


january_2024_forecast = df_new.copy()

# Подготовка данных для модели случайного леса
X = df_big.drop(['qnty'], axis=1)  
y = df_big['qnty']  

# Разделение данных на обучающий и тестовый наборы
train_size = int(len(X) * 0.8)  
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Обучение модели 
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Прогнозирование продаж на январь 2024 года
predictions = model.predict(X_test)

# Создание DataFrame
january_2024 = pd.DataFrame({'прогноз': predictions})

# Запись прогноза в CSV
january_2024_forecast.to_csv('random_forest_jan_2024.csv',sep = ';', index=False, header=False, lineterminator=';\n')

# Оценка качества прогноза с использованием RMSE
rmse = sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE прогноза на январь 2024: {rmse}')

Вывод: Для данной задачи лучше подходит модель Random Forest так как у нее RMSE меньше